In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

- it is a generative classification
- P( y = k | x = x') = P(y) * P( x = x' | y = k) #posterior = prior * likelihood

In [2]:
num_rows = 1000 

# Create a DataFrame with specified number of rows
table = pd.DataFrame(index=range(num_rows), columns=['x1', 'x2', 'y'])

# Generate random integers using apply() and a lambda function
table['x1'] = table.apply(lambda _: np.random.randint(0, 2), axis=1)
table['x2'] = table.apply(lambda _: np.random.randint(0, 2), axis=1)
table['y'] = table.apply(lambda _: np.random.randint(0, 2), axis=1)
table

,x1,x2,y
0,0,1,1
1,0,0,1
2,1,0,1
3,1,0,1
4,0,1,0
...,...,...,...
995,1,1,0
996,1,0,0
997,0,0,1
998,1,0,0


In [3]:
# Defining target and features
X = table.drop('y', axis=1)  # Replace 'TargetColumn' with the name of your target column
y = table['y']

In [21]:
# X.iloc[:,:1]
X.iloc[:,0].value_counts().get(1,0)

511

In [34]:
class NaiveBayes:
    def __init__(self, X, y, a):
        self.X = X
        self.y = y
        self.a = a #laplace smoothing constant

    def data_split(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3, shuffle = True)
        
    def split_class(self):
        self.X_0 = self.X_train[self.y_train == 0]
        self.X_1 = self.X_train[self.y_train == 1]

    def prior(self):
        self.prior_0 = len(self.X_0) / len(self.X_train)
        self.prior_1 = len(self.X_1) / len(self.X_train)
    
    def likelihood(self, X):
        Theta_0_1 = (X.iloc[:,0].value_counts().get(1, 0) + self.a) / (len(X) + self.a * len(set(self.y_train)))
        Theta_0_0 = (X.iloc[:,0].value_counts().get(0, 0) + self.a) / (len(X) + self.a * len(set(self.y_train)))
        Theta_1_1 = (X.iloc[:,1].value_counts().get(1, 0) + self.a) / (len(X) + self.a * len(set(self.y_train)))
        Theta_1_0 = (X.iloc[:,1].value_counts().get(0, 0) + self.a) / (len(X) + self.a * len(set(self.y_train)))

        return Theta_0_1 * Theta_0_0 * Theta_1_1 * Theta_1_0
    
    def posterior(self, prior, X):
        return prior * self.likelihood(X)


    def predict(self):

        self.error_count = 0

        for sample, target in zip(self.X_test, self.y_test):
            py0 = self.posterior(self.prior_0, self.X_0)
            py1 = self.posterior(self.prior_1, self.X_1)

            
            print('P(y=0 | %s) = %3.f' % (sample, py0 * 100))
            print('P(y=1 | %s) = %3.f' % (sample, py1 * 100))
            
            print(sample, target)

            # if np.argmax([py0,py1]) != target:
            #     self.error_count += 1
        
            print(" Model predicted class {} and the truth was: {} \n".format(np.argmax([py0,py1]), target))

        # print(self.error_count)
        # print(len(self.X))

    def fit(self):
        self.data_split()
        self.split_class()
        self.prior()
        self.predict()
        
        

In [35]:
nb = NaiveBayes(X, y, 5)

In [36]:
nb.fit()

P(y=0 | x1) =   0
P(y=1 | x1) =   0
x1 1
 Model predicted class 1 and the truth was: 1 

P(y=0 | x2) =   0
P(y=1 | x2) =   0
x2 0
 Model predicted class 1 and the truth was: 0 

